In [ ]:
pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 826.4 kB/s eta 0:00:00


In [ ]:
# prompt: mount drive

#from google.colab import drive
#drive.mount('/content/drive')


In [ ]:
import openai

# Set the API key directly

openai.api_key = "use your own"

In [ ]:
!pip install tabulate



# Final Code:

In [ ]:
import openai
import os
import nbformat
from nbconvert import PythonExporter
from tabulate import tabulate
import re


# Function to preprocess and check the code
def preprocess_code(code):
    # Check for syntax errors, PEP 8 compliance, etc.
    # For simplicity, assume the code is already clean and correct
    return code

# Function to generate feedback using GPT
def generate_feedback(code, question, rubric):
    # Use OpenAI GPT model to analyze the code
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a code evaluator."},
            {"role": "user", "content": f"Evaluate the following Python code based on this rubric and provide a detailed evaluation, including correctness, efficiency, readability, code quality, and relevance to the question. The evaluation should be in the format:\n\n- Correctness: <score>/20\n<comments>\n\n- Efficiency: <score>/10\n<comments>\n\n- Readability: <score>/10\n<comments>\n\n- Code Quality: <score>/10\n<comments>\n\n- Relevance: <score>/50\n<comments>\n\n Total Score: <score>/100\n\nQuestion:\n{question}\n\nCode:\n{code}\n\nRubric:\n{rubric}"}
        ]
    )
    feedback = response.choices[0].message['content'].strip()

    return feedback

# Function to extract the grades and comments from the feedback
def extract_grades_and_comments(feedback):
    # Use regular expressions to extract scores and comments
    correctness_match = re.search(r'- Correctness: (\d+)/20\n(.*?)\n\n', feedback, re.DOTALL)
    efficiency_match = re.search(r'- Efficiency: (\d+)/10\n(.*?)\n\n', feedback, re.DOTALL)
    readability_match = re.search(r'- Readability: (\d+)/10\n(.*?)\n\n', feedback, re.DOTALL)
    code_quality_match = re.search(r'- Code Quality: (\d+)/10\n(.*?)\n\n', feedback, re.DOTALL)
    relevance_match = re.search(r'- Relevance: (\d+)/50\n(.*?)\n\n', feedback, re.DOTALL)

    if not all([correctness_match, efficiency_match, readability_match, code_quality_match, relevance_match]):
        print("Feedback parsing issue. Full feedback text:")
        print(feedback)
        return 0, 0, 0, 0, 0, 0, "No comments", "No comments", "No comments", "No comments", "No comments"

    correctness_score = int(correctness_match.group(1)) if correctness_match else 0
    correctness_comments = correctness_match.group(2).strip() if correctness_match else "No comments"

    efficiency_score = int(efficiency_match.group(1)) if efficiency_match else 0
    efficiency_comments = efficiency_match.group(2).strip() if efficiency_match else "No comments"

    readability_score = int(readability_match.group(1)) if readability_match else 0
    readability_comments = readability_match.group(2).strip() if readability_match else "No comments"

    code_quality_score = int(code_quality_match.group(1)) if code_quality_match else 0
    code_quality_comments = code_quality_match.group(2).strip() if code_quality_match else "No comments"

    relevance_score = int(relevance_match.group(1)) if relevance_match else 0
    relevance_comments = relevance_match.group(2).strip() if relevance_match else "No comments"

    total_score = correctness_score + efficiency_score + readability_score + code_quality_score + relevance_score

    related = "Yes" if relevance_score > 0 else "No"

    if relevance_score == 0:
        total_score = 0

    return (correctness_score, efficiency_score, readability_score, code_quality_score, relevance_score, total_score,
            correctness_comments, efficiency_comments, readability_comments, code_quality_comments, relevance_comments, related)

# Function to check if a cell contains executable code
def is_executable_code(cell_code):
    lines = cell_code.split('\n')
    for line in lines:
        line = line.strip()
        if line and not line.startswith('#'):
            return True
    return False

# Function to read and process a Python file
def read_python_file(file_path):
    with open(file_path, 'r') as file:
        code = file.read()
    return [code]  # Return as a list to maintain consistency

# Function to read and process a Jupyter notebook file
def read_notebook_file(file_path):
    with open(file_path, 'r') as file:
        nb = nbformat.read(file, as_version=4)
    cells = nb.cells
    return cells

# Main function to handle user input and grading
def main():
    # Get file path from user input
    file_path = input("Enter the path to the Python (.py) or Jupyter notebook (.ipynb) file: ").strip()

    # Check the file extension and read the file accordingly
    if file_path.endswith('.py'):
        code_cells = read_python_file(file_path)
        questions = [""] * len(code_cells)  # No questions for .py files
    elif file_path.endswith('.ipynb'):
        cells = read_notebook_file(file_path)
        code_cells = []
        questions = []
        for cell in cells:
            if cell.cell_type == 'markdown':
                questions.append(cell['source'].strip())
            elif cell.cell_type == 'code' and is_executable_code(cell['source']):
                code_cells.append(cell['source'].strip())
                if len(questions) > len(code_cells):
                    questions.pop()  # Remove the question if it doesn't have a corresponding code cell
    else:
        print("Unsupported file format. Please provide a .py or .ipynb file.")
        return

    # Example rubric
    rubric = """
    1. Correctness: Does the code produce the expected output? (20%)
    2. Efficiency: Is the code optimized for performance? (10%)
    3. Readability: Is the code well-documented and easy to understand? (10%)
    4. Code Quality: Does the code follow best practices and coding standards? (10%)
    5. Relevance: Is the code relevant to the question? (50%)
    """

    total_cells = len(code_cells)
    if total_cells == 0:
        print("No code cells found to grade.")
        return

    grade_per_cell = 100 / total_cells  # Adjusted to be out of 100
    total_grade = 0
    overall_feedback = []

    table_data = []

    for i, (question, cell_code) in enumerate(zip(questions, code_cells)):
        clean_code = preprocess_code(cell_code)
        feedback = generate_feedback(clean_code, question, rubric)

        # Extract grades and comments
        (correctness_score, efficiency_score, readability_score, code_quality_score, relevance_score, cell_total_score,
         correctness_comments, efficiency_comments, readability_comments, code_quality_comments, relevance_comments, related) = extract_grades_and_comments(feedback)

        # Check if feedback was generated correctly
        if cell_total_score == 0 and any([correctness_comments, efficiency_comments, readability_comments, code_quality_comments, relevance_comments]):
            print(f"Warning: Cell {i+1} received a score of 0 but contains feedback.")
            print(f"Feedback: {feedback}")

        # Format the feedback according to the fixed format
        formatted_feedback = (
            f"- Correctness: {correctness_score}/20\n"
            f"{correctness_comments}\n\n"
            f"- Efficiency: {efficiency_score}/10\n"
            f"{efficiency_comments}\n\n"
            f"- Readability: {readability_score}/10\n"
            f"{readability_comments}\n\n"
            f"- Code Quality: {code_quality_score}/10\n"
            f"{code_quality_comments}\n\n"
            f"- Relevance: {relevance_score}/50\n"
            f"{relevance_comments}\n\n"
            f"Total Score: {cell_total_score}/100"
        )

        # Adjust the grade to be out of the grade per cell
        if relevance_score == 0:
            adjusted_grade = 0
        else:
            adjusted_grade = (cell_total_score / 100) * grade_per_cell

        total_grade += adjusted_grade
        overall_feedback.append(formatted_feedback)

        table_data.append([f"Cell {i+1}", formatted_feedback, f"{adjusted_grade:.2f}/{grade_per_cell:.2f}", related])

    # Print the table
    print(tabulate(table_data, headers=["Evaluation Based on Rubric", "Comments", "Grade", "Related to Question"], tablefmt="grid"))

    # Print the total grade
    print(f"\nTotal Grade: {total_grade:.2f}/100")

if __name__ == "__main__":
    main()


Enter the path to the Python (.py) or Jupyter notebook (.ipynb) file: /content/coding_questions.ipynb
Feedback: - Correctness: 0/20
The code provided is not correct for the given question. The code defines a function `fibonacci` that calculates the nth Fibonacci number, rather than finding the maximum value in a list of integers. The print statement also calls the `fibonacci` function instead of `find_max`.

- Efficiency: 8/10
The code for calculating Fibonacci numbers is efficient and optimized in terms of performance. It uses a loop to calculate the Fibonacci sequence up to the given value of n. However, for the given question about finding the maximum value in a list of integers, the code is inefficient and irrelevant.

- Readability: 8/10
The code is relatively readable with clear variable names and a concise implementation. However, the function `fibonacci` could benefit from some documentation to explain its purpose and inputs.

- Code Quality: 8/10
The code follows Python's best